In [1]:
import numpy as np
from scipy.stats import pareto
from evaluator import evaluator

np.random.seed(242874)
myEvaluator = evaluator()

/home/ofirblumer/.local/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Here we show how to use the evaluator class to estimate the unbiased MFPT from simulations accelerated with resetting.

First, we sample 100 trajectories with resetting from the hyperexponential distribution of the paper (Equation 6). We use a timer of $T^*=2$.

In [2]:
exact = 5.005

p = 0.5
l2 = 0.00001
l1 = l2 * 1000
b = np.linspace(0.5, 1000000.5, 1000000)
CDF = 1 - (1 - p) * np.exp(-l1 * b) - p * np.exp(-l2 * b)
density = CDF[1:] - CDF[:-1]
density = density / sum(density)
bins = np.linspace(1, 1000000, 999999)

fpts = []
endTimes = []
count = 0
timer = 2000
time = 0
while count < 100:
    new = np.random.choice(bins, p = density)
    endTimes.append(new)
    time += min([new, timer])
    if new < timer:
        count += 1
        fpts.append(time)
        time = 0
MFPT = np.mean(fpts) / 10000
print("MFPT with timer of 2 is {:.3f}, a speedup of {:.2f}".format(MFPT, exact / MFPT))

MFPT with timer of 2 is 0.212, a speedup of 23.63


We obtain the estimation for the unbiased MFPT using the estimate function.

In [3]:
estimated = myEvaluator.estimate(endTimes) / 10000
error = abs(estimated - exact) / exact * 100
print("The estimated MFPT is {:.2f}, an error of {:.2f}%".format(estimated, error))

The estimated MFPT is 4.85, an error of 3.14%


By default, the function assumes that all terms in the endTimes array end in first-passage or with a timer which equals the maximum of the array.
We can also explicitly define the timer, which can be usefull when combining data obtained with different timers. Then, we can safely use the shortest of these timers. Below we give an example where we add 200 trajectories with a timer of $T^*=1.5$.

In [4]:
count = 0
timer = 1500
time = 0
while count < 200:
    new = np.random.choice(bins, p = density)
    endTimes.append(new)
    time += min([new, timer])
    if new < timer:
        count += 1
        fpts.append(time)
        time = 0

estimated = myEvaluator.estimate(endTimes, timer = timer) / 10000
error = abs(estimated - exact) / exact * 100
print("The estimated MFPT is {:.2f}, an error of {:.2f}%.".format(estimated, error))

The estimated MFPT is 3.97, an error of 20.65%.


We can also retrieve the estimated $t'$ and $k$.

In [5]:
params = myEvaluator.estimate(endTimes, timer = timer, returnParams = True)
print(r"The estimated MFPT is {:.2f}, k is {:.2f}, and t' is {:.2f}".format(params["MFPT"] / 10000, params["k"] * 10000, params["tPrime"] / 10000))

The estimated MFPT is 3.97, k is 0.13, and t' is 0.05


By default, the function uses at least 5 terms for the linear fit of the survival function. We can change this parameter, for instance, to 10 values.
We can also obtain a table of all the calculated linear fits for all choices of $t'$, along with their $R^2$ values.

In [6]:
myEvaluator.estimate(endTimes, timer = timer, returnTable = True, minSamples = 10)[-10:]

,tPrime,slope,R
281,298.000297,-0.000046,0.622605
282,300.000299,-0.000044,0.638812
283,303.000302,-0.000041,0.655572
284,307.000306,-0.000039,0.673072
285,314.000313,-0.000036,0.691498
286,316.000315,-0.000034,0.711958
287,324.000323,-0.000031,0.731939
288,328.000327,-0.000029,0.753347
289,341.000340,-0.000027,0.773228
290,358.000357,-0.000024,0.794298


Finally, the default assumption for the tail is an exponential. We can also assume a power law, as we demonstrate for the pareto distribution (Equation 7) with a timer of $T^*=2$.

In [7]:
exact = 5

fpts = []
endTimes = []
count = 0
timer = 2
time = 0
while count < 100:
    new = pareto.rvs(b=1.25)
    endTimes.append(new)
    time += min([new, timer])
    if new < timer:
        count += 1
        fpts.append(time)
        time = 0
        
MFPT = np.mean(fpts)
estimated = myEvaluator.estimate(endTimes, tail = "power")
error = abs(estimated - exact) / exact * 100
print("MFPT with timer of 2 is {:.3f}, a speedup of {:.2f}".format(MFPT, exact / MFPT))
print("The estimated MFPT is {:.2f}, an error of {:.2f}%".format(estimated, error))

MFPT with timer of 2 is 2.774, a speedup of 1.80
The estimated MFPT is 3.67, an error of 26.52%
